In [0]:
PATH = '/content/drive/My Drive/Colab Notebooks/House Price Predictor /data/'

**IMPORTING LIBRARIES**

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.losses import SparseCategoricalCrossentropy
from keras.losses import MeanSquaredError
from keras.layers import Conv2D, MaxPool2D

VOC_SIZE = 500
INPUT_SHAPE = 75

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


**DATA PREPRATION**

In [0]:
df_train = pd.read_csv(PATH+'train.csv')
df_test = pd.read_csv(PATH+'test.csv')
df_test['SalePrice'] = np.zeros(len(df_test))

df = df_train.append(df_test)

df['LotFrontage']=df['LotFrontage'].fillna(df['LotFrontage'].mean())

df['BsmtCond']=df['BsmtCond'].fillna(df['BsmtCond'].mode()[0])
df['BsmtQual']=df['BsmtQual'].fillna(df['BsmtQual'].mode()[0])
df['FireplaceQu']=df['FireplaceQu'].fillna(df['FireplaceQu'].mode()[0])
df['GarageType']=df['GarageType'].fillna(df['GarageType'].mode()[0])
df['GarageFinish']=df['GarageFinish'].fillna(df['GarageFinish'].mode()[0])
df['GarageQual']=df['GarageQual'].fillna(df['GarageQual'].mode()[0])
df['GarageCond']=df['GarageCond'].fillna(df['GarageCond'].mode()[0])

df.drop(['Id', 'Alley', 'PoolQC','Fence','MiscFeature'],axis=1,inplace=True)

# df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

X = df.iloc[:,:-1] 
y = np.array(df.iloc[:,-1], dtype='float64')

categorical_cols = X.select_dtypes('object').columns.to_list() 
for i in categorical_cols:
  for j in range(len(X)):
    X[i].iloc[j] = one_hot(str(X[i].iloc[j]), VOC_SIZE)[0]
    
scaler = MinMaxScaler((0,1))
X = scaler.fit_transform(np.array(X, dtype='float64'))

X_train = X[:1400,]
y_train = y[:1400,]
X_val = X[1400:1460,]
y_val = y[1400:1460,]
X_test = X[1460:,]
y_test = y[1460:,]


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


"\nsent_length=81\nX_train = pad_sequences(X_train,padding='pre',maxlen=sent_length)\nX_test = pad_sequences(X_test,padding='pre',maxlen=sent_length)\n\nX_train = X_train.reshape(len(X_train), 9, 9, 1)\nX_test = X_test.reshape(len(X_test), 9, 9, 1)\n\n"

DNN MODEL

In [0]:
dnn_model = Sequential()

dnn_model.add(Dense(50, input_shape=(INPUT_SHAPE,))) 
dnn_model.add(Dense(25, init='he_uniform', activation='relu'))   
dnn_model.add(Dense(25, init='he_uniform', activation='relu'))  
dnn_model.add(Dense(50, init='he_uniform', activation='relu'))
dnn_model.add(Dense(1, init = 'he_uniform')) 

dnn_model.compile(optimizer='Adamax', loss='mean_squared_error', metrics=['mean_squared_error'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(25, activation="relu", kernel_initializer="he_uniform")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(25, activation="relu", kernel_initializer="he_uniform")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="he_uniform")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="he_uniform")`
  import sys


In [0]:
dnn_model.fit(X_train, y_train, validation_data=(X_val, y_val),batch_size = 10, epochs=500)

In [0]:
i = 19
print(f"Predicted : {dnn_model.predict(X_val)[i][0]} , Actual : {y_val[i]} , Difference : {abs(dnn_model.predict(X_val)[i][0] - y_val[i])}\n")

val_loss, val_acc = dnn_model.evaluate(X_val, y_val)
print(f"Val Loss : {val_loss}, Val Acc : {val_acc}")

Predicted : 216873.15625 , Actual : 223000.0 , Difference : 6126.84375

60/60 [==============================] - 0s 81us/step
Val Loss : 881830101.3333334, Val Acc : 881830080.0


In [0]:
y = dnn_model.predict(X_test)

In [0]:
submission = pd.DataFrame({ 'Id': [i for i in range(1461,1461 + len(y))], 'SalePrice': [i[0] for i in y] })
submission.to_csv("sample_submission_REG.csv", index=False)